##JobsDataAnalysisProject

Идея:

1) Собрать с основных мировых рынков труда информацию о должности и о требованиях к кандидату. Кроме того сгруппировать все вакансии по industry name, так как существуют общие скилы, требуемые почти во всех вакансиях.

[Общий список вакансий рынка](#scrollTo=CeuOgT02tEOM&line=5&uniqifier=1)

Обучаем rnn сеть на описаниях, на выходе one hot соответствие описания должности на рынке.

2) С netology.ru спарсить образовательные программы, по каждой из них получаем скилы, которые получит участник после прохождения.

[Образовательные программы](#scrollTo=xAIQD1xn5wuv&line=2&uniqifier=1)

3) Клиент в свободной форме пишет от лица работодателя работы своей мечты такое описание вакансии, чтобы сам этот клиент, по его мнению ей соответствовал. Подаём на вход сети из пункта 2 и получаем, по мнению клиента, лучшую должность для него.

4) Для этой должности забираем необходимые скилы взятые по всем вакансиям из исследования [WorldBankGroup по linkedIn](#scrollTo=nngmdjXCLQwo&line=1&uniqifier=1). Набор скилов дополняем средним по всем должностям для шума и частичных совпадений.

5) Запрашиваем у клиента страну будущей работы. С CourseraGSI dataset [получаем ситуацию по нужным скилам в данной стране](#scrollTo=qxyxcN61K_zI&line=1&uniqifier=1). В зависимости от ситуации с нужными скилами в стране, даем оценку, нужно ли нам сильно их поднимать. Или текущего пусть даже низкого уровня хватит

6) По маске скилов находим подборку нетологии, остальное дополняем бесплатными или реферальными с [CourseTalk отзывов студентов](#scrollTo=gANiqgivKmdq&line=1&uniqifier=1)

upd. Время закончилось на настройке lstm )


## Setup

In [ ]:
!pip3 install scv

     |████████████████████████████████| 7.2MB 7.5MB/s 
     |████████████████████████████████| 348kB 38.3MB/s 


In [ ]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from babel import Locale
locale = Locale('en', 'US')

CURRENCY_SYMBOLS = [''.join(filter(lambda x: not x.isalpha(), st)) for st in list(locale.currency_symbols.values())]

print(CURRENCY_SYMBOLS)

INDUSTRY_NAMES = conf('INDUSTRY_NAMES')

['$', '$', '$', '¥', '€', '£', '$', '₪', '₹', '¥', '₩', '$', '$', '$', '$', '₫', '', '$', '', '', '¤']


## Import

In [ ]:
from __future__ import print_function
import os
import numpy as np
from google.colab import drive
from tqdm.autonotebook import tqdm
import pandas as pd

import requests as rq
from datetime import datetime
import csv
import json

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """


In [ ]:
def driveReconnect():
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/MCH/data')
  os.listdir()

driveReconnect()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Library

In [ ]:
def log(toLog = "", status = "log"):
    lg = open("log.txt", "a")
    currentTime = datetime.now().strftime("%H:%M:%S")
    errorSymbol = '!'
    rTimes = 0
    if status == "err":
        rTimes = 10
    lg.write(errorSymbol * rTimes + currentTime + " : " + toLog + "\n")

In [ ]:
def conf(name, value = None):
    if value is not None:
        confFile = open('./configData.json', 'r')
        config = json.loads(confFile.read())
        confFile.close()

        config.update({name : value})

        confFile = open('./configData.json', 'w')
        confFile.write(json.dumps(config))
        confFile.close()

        return value
    else:
        confFile = open('./configData.json', 'r')
        config = json.loads(confFile.read())
        confFile.close()

        return config.get(name, None)

In [ ]:
def getNetologyCourseInfo(url):
    if url == None:
        return url
    
    cName = url.split('/')[-1]

    rs = rq.get("https://netology.ru/backend/api/page_contents/programs%2F" + cName)

    courseInfo = []

    if rs.status_code == 200:
        try:
            courseInfo = rs.json()
        except:
            log('rs.content is ' + str(rs.content), 'err')    
    else:
        log('re.status_code is ' + str(rs.status_code), 'err')
        return

    skills = []

    try:
        skills = courseInfo["content"]["resume"]["skills"]
        return list(map(getPureSkillFromText, skills))
    except:
        try:
            skills = [t["title"] for t in courseInfo["content"]["resume"]["technologies"]]
            return skills
        except:
            return []

In [ ]:
def updateNetologyCoursesData():
    rs = rq.get("https://netology.ru/backend/api/programs")

    courses = []

    if rs.status_code == 200:
        courses = rs.json()
    else:
        log('re.status_code is ' + str(rs.status_code), 'err')
        return

    with open('./Datasets/NetologyCoursesDataset.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)

        header = ['id', 'type', 'name', 'url', 'duration', 'projects_amount', 'date', 'main_direction', 'directions', 'skills', 'price_type', 'description']
        writer.writerow(header)

        for course in tqdm(courses):
            writer.writerow(
                [course.get(key, None) if key != 'skills' else getNetologyCourseInfo(course.get('url')) for key in header]
            )

    display(pd.read_csv('./Datasets/NetologyCoursesDataset.csv').head())

updateNetologyCoursesData()

,id,type,name,url,duration,projects_amount,date,main_direction,directions,skills,price_type,description
0,21275,profession,Тестировщик ПО,//netology.ru/programs/qa,7 месяцев,10 занятий,2021-07-01,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...","['Java', 'GitHub', 'Git', 'Selenium', 'SQL', '...",paid,"<p>Станьте IT-специалистом, отвечающим за каче..."
1,21295,profession,Инженер по тестированию: с нуля до middle,//netology.ru/programs/qa-middle,14 месяцев,10 занятий,2021-07-01,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...","['<p>Git</p>', '<p>Java</p>', '<p>Selenium</p>...",paid,"<p>Станьте IT-специалистом, отвечающим за каче..."
2,20528,profession,Интернет-маркетолог: с нуля до middle,//netology.ru/programs/distance-course-interne...,10 месяцев,61 занятие,2021-06-28,"{'slug': 'marketing', 'name': 'Маркетинг'}","[{'slug': 'marketing', 'name': 'Маркетинг'}]",[],paid,"<p>Основы маркетинга, анализ и сегментация цел..."
3,19718,profession,Python-разработчик с нуля,//netology.ru/programs/python,8 месяцев,26 занятий,2021-06-15,"{'slug': 'development', 'name': 'Программирова...","[{'slug': 'development', 'name': 'Программиров...","['<font color=""#000000"">Python</font>', '<p><f...",paid,"<p>Став Python-разработчиком, вы делаете первы..."
4,12701,profession,SMM-менеджер: продвижение бизнеса в социальных...,//netology.ru/programs/smm-specialist,8 месяцев,128 занятий,2021-06-28,"{'slug': 'marketing', 'name': 'Маркетинг'}","[{'slug': 'marketing', 'name': 'Маркетинг'}]","['Popsters', 'JagaJam', 'Target Hunter', 'Pepp...",paid,"<p>Социальные сети для бизнеса, построение стр..."


In [ ]:
def getSkillsFromText(text):
    return []

In [ ]:
def getPriceFromText(text):
    
    text = list(str(text))

    price = "".join(filter(lambda x: x.isdecimal() or x in CURRENCY_SYMBOLS or x in ['.', ','], text))

    if len(price) > 0:
        if price[0] == '.' or price[0] == ',':
            price = price[1:]
        else:
            if price[-1] == '.' or price[-1] == ',':
                price = price[:-1]
    return price

In [ ]:
def getHighlightsFromText(text):
    return ""

In [ ]:
def getPureSkillFromText(text):
    return text

## Datasets manipulation


~load

In [ ]:
jobPostingsDatasetNaukriClean = pd.read_csv('./Datasets/JobPostingsDatasetClean(Naukri.com).csv')
jobPostingsDatasetNaukriClean.head(5)

,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,2019-07-05 01:46:07 +0000,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager
1,3c52d436e39f596b22519da2612f6a56,2019-07-06 08:04:50 +0000,Online Bidding Executive,Not Disclosed by Recruiter,2 - 5 yrs,pre sales| closing| software knowledge| clien...,Retail Sales,"Pune,Pune","Sales , Retail , Business Development","IT-Software, Software Services",Sales Executive/Officer
2,ffad8a2396c60be2bf6d0e2ff47c58d4,2019-08-05 15:50:44 +0000,Trainee Research/ Research Executive- Hi- Tec...,Not Disclosed by Recruiter,0 - 1 yrs,Computer science| Fabrication| Quality check|...,R&D,Gurgaon,"Engineering Design , R&D","Recruitment, Staffing",R&D Executive
3,7b921f51b5c2fb862b4a5f7a54c37f75,2019-08-05 15:31:56 +0000,Technical Support,"2,00,000 - 4,00,000 PA.",0 - 5 yrs,Technical Support,Admin/Maintenance/Security/Datawarehousing,Mumbai,"IT Software - Application Programming , Mainte...","IT-Software, Software Services",Technical Support Engineer
4,2d8b7d44e138a54d5dc841163138de50,2019-07-05 02:48:29 +0000,Software Test Engineer -hyderabad,Not Disclosed by Recruiter,2 - 5 yrs,manual testing| test engineering| test cases|...,Programming & Design,Hyderabad,IT Software - QA & Testing,"IT-Software, Software Services",Testing Engineer


In [ ]:
jobPostingsDatasetReedUk = pd.read_csv('./Datasets/JobPostingsDataset(Reed.uk).csv')
jobPostingsDatasetReedUk.head(5)

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
0,catering jobs,Chudleigh,Haulfryn Group,uk,reed,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,"Permanent, full-time",3/12/2018,"£18,323 per annum",Devon
1,law jobs,Swindon,Reed,uk,reed,Apply now This role requires an experienced s...,NaN,Corporate Legal Secretary,"Permanent, full-time",3/8/2018,"£18,000 - £22,000 per annum",Wiltshire
2,factory jobs,Norfolk,Swanstaff Recruitment Ltd,uk,reed,Apply now Job Title: Warehouse personLocation...,NaN,Warehouse Person,"Permanent, full-time",2/27/2018,Salary not specified,East Anglia
3,law jobs,Hurstpierpoint,Castles Solicitors,uk,reed,Apply now Castles Solicitors are looking for ...,Required skills Admin Case Management Corresp...,Legal Secretary / Admin Assistant,"Permanent, full-time or part-time",2/21/2018,"£15,000 - £17,000 per annum, pro-rata",West Sussex
4,factory jobs,Hertfordshire,Marketplace Group,uk,reed,Apply now International Pharmaceutical Manufa...,NaN,Category Planner - Pharmaceutical,"Permanent, full-time",3/1/2018,"£37,000 - £45,000 per annum",South East England


In [ ]:
jobPostingsDatasetNaukriMessy = pd.read_csv('./Datasets/JobPostingsDatasetMessy(Naukri.com).csv')
jobPostingsDatasetNaukriMessy.head(5)

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8
1,find live infotech,UG: B.Tech/B.E. - Any Specialization PG:MBA/PG...,0 - 0 yrs,Advertising / PR / MR / Event Management,Job Description Send me Jobs like this Quali...,210516002391,Chennai,Work Based Onhome Based Part Time.,60.0,"1,50,000 - 2,50,000 P.A. 20000",2016-05-21 19:30:00 +0000,NaN,Marketing,d4c72325e57f89f364812b5ed5a795f0
2,Softtech Career Infosystem Pvt. Ltd,UG: Any Graduate - Any Specialization PG:Any P...,4 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this - as ...,101016900534,Bengaluru,Pl/sql Developer - SQL,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,c47df6f4cfdf5b46f1fd713ba61b9eba
3,Onboard HRServices LLP,UG: Any Graduate - Any Specialization PG:CA Do...,11 - 15 yrs,Banking / Financial Services / Broking,Job Description Send me Jobs like this - Inv...,81016900536,"Mumbai, Bengaluru, Kolkata, Chennai, Coimbator...",Manager/ad/partner - Indirect Tax - CA,NaN,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,Accounts,115d28f140f694dd1cc61c53d03c66ae
4,Spire Technologies and Solutions Pvt. Ltd.,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,6 - 8 yrs,IT-Software / Software Services,Job Description Send me Jobs like this Pleas...,120916002122,Bengaluru,JAVA Technical Lead (6-8 yrs) -,4.0,Not Disclosed by Recruiter,2016-10-13 16:20:55 +0000,NaN,IT Software - Application Programming,a12553fc03bc7bcced8b1bb8963f97b4


In [ ]:
globalCoursesInfoDataset = pd.read_csv('./Datasets/CoursesInfoDataset(CourseTalk.com).csv')
globalCoursesInfoDataset.head(10)

,CourseTitle,Description,Availability,Cost
0,C# Basics,In this course you will gain all the knowledge...,On demand,$25/mo
1,Java Basics,In this course you will gain all the knowledge...,On demand,$25/mo
2,Mindfulness for Wellbeing and Peak Performance,Learn mindfulness techniques to reduce stress ...,Upcoming,Free
3,Introduction to Probability - The Science of U...,"The world is full of uncertainty: accidents, s...",On demand,Free
4,Existential Well-being Counseling: A Person-ce...,This humanities course offers you the opportun...,On demand,Free
5,sec.edu - Security Engineering - Applied Cyber...,UNSW in proud partnership with the CBA are rel...,On demand,Free
6,Foundations of Computer Science for Teachers,"Thinking about teaching Computer Science, or i...",On demand,Free
7,The Ancient Greek Hero in 24 Hours,Explore what it means to be human today by stu...,On demand,Free
8,The Science of Gastronomy,This course introduces a number of basic scien...,Upcoming,Free
9,Chinese Thought: Ancient Wisdom Meets Modern S...,This course is designed to give students a tho...,Upcoming,Free


In [ ]:
courseraGSIDataset = pd.read_csv('./Datasets/CourseraGSI.csv')
courseraGSIDataset.head(10)

,country,iso3,region,incomegroup,iso,competency_id,percentile_rank,percentile_category
0,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,statistical-programming,0.864407,Cutting-edge (75%-100%)
1,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,statistics,0.237288,Lagging (0-25%)
2,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,machine-learning,0.355932,Emerging (25%-50%)
3,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,software-engineering,0.322034,Emerging (25%-50%)
4,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,fields-of-mathematics,0.491525,Emerging (25%-50%)
5,United Arab Emirates,ARE,Middle East & North Africa,High income,AE,artificial-intelligence,0.186441,Lagging (0-25%)
6,Argentina,ARG,Latin America & Caribbean,Upper middle income,AR,fields-of-mathematics,0.355932,Emerging (25%-50%)
7,Argentina,ARG,Latin America & Caribbean,Upper middle income,AR,statistical-programming,0.576271,Competitive (50%-75%)
8,Argentina,ARG,Latin America & Caribbean,Upper middle income,AR,software-engineering,0.559322,Competitive (50%-75%)
9,Argentina,ARG,Latin America & Caribbean,Upper middle income,AR,statistics,0.610169,Competitive (50%-75%)


In [ ]:
skillsIndustryNeedsDatasetWorldBankGroup = pd.read_csv('./Datasets/IndustrySkillsNeeds(WorldBankGroup).csv', delimiter=';', encoding = 'latin-1')

INDUSTRY_NAMES = conf("INDUSTRY_NAMES", list(set(skillsIndustryNeedsDatasetWorldBankGroup['industry_name'])))

skillsIndustryNeedsDatasetWorldBankGroup.head(10)

,isic_section_index,isic_section_name,industry_name,skill_group_category,skill_group_name,skill_group_rank
0,B,Mining and quarrying,Mining & Metals,Specialized Industry Skills,Mining,1
1,B,Mining and quarrying,Mining & Metals,Soft Skills,Negotiation,2
2,B,Mining and quarrying,Mining & Metals,Business Skills,Project Management,3
3,B,Mining and quarrying,Mining & Metals,Business Skills,Business Management,4
4,B,Mining and quarrying,Mining & Metals,Specialized Industry Skills,Earth Science,5
5,B,Mining and quarrying,Mining & Metals,Business Skills,Manufacturing Operations,6
6,B,Mining and quarrying,Mining & Metals,Specialized Industry Skills,Drilling Engineering,7
7,B,Mining and quarrying,Mining & Metals,Tech Skills,Digital Literacy,8
8,B,Mining and quarrying,Mining & Metals,Specialized Industry Skills,Maintenance & Repair,9
9,B,Mining and quarrying,Mining & Metals,Business Skills,Procurement,10


In [ ]:
skillsPenetrationDatasetWorldBankGroup = pd.read_csv('./Datasets/UseSkillPenetration(World Bank Group).csv', delimiter=';', encoding = 'latin-1')
skillsPenetrationDatasetWorldBankGroup.head(10)

,skill_group_category,skill_group_name,isic_section_index,isic_section_name,industry_name,skill_group_penetration_rate
0,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Accounting,"0,00719"
1,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Law Practice,"0,00244"
2,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Executive Office,"0,00222"
3,Business Skills,Accounts Payable,C,Manufacturing,Packaging & Containers,"0,00132"
4,Business Skills,Accounts Payable,B,Mining and quarrying,Oil & Energy,"0,00132"
5,Business Skills,Accounts Payable,C,Manufacturing,Printing,"0,00128"
6,Business Skills,Accounts Payable,C,Manufacturing,Machinery,"0,00112"
7,Business Skills,Accounts Payable,K,Financial and insurance activities,Venture Capital & Private Equity,"0,00102"
8,Business Skills,Accounts Payable,C,Manufacturing,Plastics,"9,40E-04"
9,Business Skills,Accounts Payable,M,Professional scientific and technical activities,Legal Services,"9,20E-04"


~set structure

In [ ]:
jobPostingsDatasets = {
    "naukri1" : jobPostingsDatasetNaukriClean,
    "reedUk" : jobPostingsDatasetReedUk,
    "naukri2" : jobPostingsDatasetNaukriMessy
}

~modify

In [ ]:
jobPostingsDatasetNaukriClean.head(1)

,Uniq Id,Crawl Timestamp,Job Title,Job Salary,Job Experience Required,Key Skills,Role Category,Location,Functional Area,Industry,Role
0,9be62c49a0b7ebe982a4af1edaa7bc5f,2019-07-05 01:46:07 +0000,Digital Media Planner,Not Disclosed by Recruiter,5 - 10 yrs,Media Planning| Digital Media,Advertising,Mumbai,"Marketing , Advertising , MR , PR , Media Plan...","Advertising, PR, MR, Event Management",Media Planning Executive/Manager


In [ ]:
jobPostings1 = jobPostingsDatasetNaukriClean
jobPostings1 = jobPostings1[['Job Title', 'Key Skills', 'Job Salary', 'Functional Area', 'Role Category']]

jobPostings1['Job Salary'] = list(map(getPriceFromText, jobPostings1['Job Salary']))
jobPostings1.rename(columns={"Job Salary": "salary"}, inplace=True)

jobPostings1.rename(columns={"Functional Area": "description_highlights"}, inplace=True)

jobPostings1.rename(columns={"Key Skills": "skills"}, inplace=True)

jobPostings1.rename(columns={"Role Category": "industry"}, inplace=True)

jobPostings1.rename(columns={"Job Title": "job_title"}, inplace=True)

jobPostings1.head(1)

,job_title,skills,salary,description_highlights,industry
0,Digital Media Planner,Media Planning| Digital Media,,"Marketing , Advertising , MR , PR , Media Plan...",Advertising


In [ ]:
jobPostingsDatasetReedUk.head(1)

,category,city,company_name,geo,job_board,job_description,job_requirements,job_title,job_type,post_date,salary_offered,state
0,catering jobs,Chudleigh,Haulfryn Group,uk,reed,Apply now New opportunity not to be missed! H...,NaN,Commis Chef,"Permanent, full-time",3/12/2018,"£18,323 per annum",Devon


In [ ]:
jobPostings2 = jobPostingsDatasetReedUk
jobPostings2 = jobPostings2[['category', 'job_title', 'job_description', 'salary_offered', 'job_requirements']]

jobPostings2['salary_offered'] = list(map(getPriceFromText, jobPostings2['salary_offered']))
jobPostings2.rename(columns={"salary_offered": "salary"}, inplace=True)

jobPostings2['job_description'] = list(map(getHighlightsFromText, jobPostings2['job_description']))
jobPostings2.rename(columns={"job_description": "description_highlights"}, inplace=True)

jobPostings2.rename(columns={"job_requirements": "skills"}, inplace=True)

jobPostings2.rename(columns={"category": "industry"}, inplace=True)

jobPostings2.head(1)

,industry,job_title,description_highlights,salary,skills
0,catering jobs,Commis Chef,,"£18,323",NaN


In [ ]:
jobPostingsDatasetNaukriMessy.head(1)

,company,education,experience,industry,jobdescription,jobid,joblocation_address,jobtitle,numberofpositions,payrate,postdate,site_name,skills,uniq_id
0,MM Media Pvt Ltd,UG: B.Tech/B.E. - Any Specialization PG:Any Po...,0 - 1 yrs,Media / Entertainment / Internet,Job Description Send me Jobs like this Quali...,210516002263,Chennai,Walkin Data Entry Operator (night Shift),NaN,"1,50,000 - 2,25,000 P.A",2016-05-21 19:30:00 +0000,NaN,ITES,43b19632647068535437c774b6ca6cf8


In [ ]:
jobPostings3 = jobPostingsDatasetNaukriMessy
jobPostings3 = jobPostings3[['jobtitle', 'jobdescription', 'payrate', 'skills', 'industry']]

jobPostings3['payrate'] = list(map(getPriceFromText, jobPostings3['payrate']))
jobPostings3.rename(columns={"payrate": "salary"}, inplace=True)

jobPostings3['jobdescription'] = list(map(getHighlightsFromText, jobPostings3['jobdescription']))
jobPostings3.rename(columns={
    "jobdescription": "description_highlights",
    "jobtitle": "job_title"
    }, inplace=True)

jobPostings3.head(1)

,job_title,description_highlights,salary,skills,industry
0,Walkin Data Entry Operator (night Shift),,"1,50,0002,25,000",ITES,Media / Entertainment / Internet


~concat

In [ ]:
jobPostings = pd.concat([jobPostings1, jobPostings2, jobPostings3], axis=0)

log("JobPosting1 shape is " + str(jobPostings1.shape))
log("JobPosting2 shape is " + str(jobPostings2.shape))
log("JobPosting3 shape is " + str(jobPostings3.shape))

log("JobPostings shape is " + str(jobPostings.shape))

In [ ]:
jobPostingsFull = jobPostings.dropna(subset=['skills'])

log("JobPostings shape is " + str(jobPostingsFull.shape))

jobPostingsFull.head(50)

,job_title,skills,salary,description_highlights,industry
0,Digital Media Planner,Media Planning| Digital Media,,"Marketing , Advertising , MR , PR , Media Plan...",Advertising
1,Online Bidding Executive,pre sales| closing| software knowledge| clien...,,"Sales , Retail , Business Development",Retail Sales
2,Trainee Research/ Research Executive- Hi- Tec...,Computer science| Fabrication| Quality check|...,,"Engineering Design , R&D",R&D
3,Technical Support,Technical Support,"2,00,0004,00,000","IT Software - Application Programming , Mainte...",Admin/Maintenance/Security/Datawarehousing
4,Software Test Engineer -hyderabad,manual testing| test engineering| test cases|...,,IT Software - QA & Testing,Programming & Design
5,Opening For Adobe Analytics Specialist,adobe experience manager| digital| digital ma...,"7,00,00017,00,000","IT Software - Application Programming , Mainte...",Programming & Design
6,Sales- Fresher-for Leading Property Consultant,channel partners| real estate| negotiation| p...,"2,00,0003,00,000","Sales , Retail , Business Development",Retail Sales
7,Opportunity For Azure Devops Architect For Hy...,TFS| Azure| Git| VSTS| Docker| DynaTrace| Spl...,,"IT Software - Application Programming , Mainte...",Programming & Design
8,BDE- PUNE,Bde,,"Sales , Retail , Business Development",Institutional Sales
9,Technical Support/ Product Support,technical support| support services| applicat...,,"ITES , BPO , KPO , LPO , Customer Service , Op...",Voice


##Train and prediction

In [ ]:
##Train and prediction

In [ ]:
from __future__ import print_function
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Bidirectional
from keras.layers import LSTM, CuDNNLSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, LambdaCallback
import numpy as np
import argparse
from google.colab import drive
import datetime
from keras.models import model_from_json
import keras
from bs4 import BeautifulSoup

tf.logging.set_verbosity(tf.logging.ERROR)

In [ ]:
def driveReconnect():
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/lstm')
  os.listdir()

driveReconnect()

In [ ]:
#for english v

##data_path = "./simple-examples/data/"

#for rus v

data_path = "./simple-examples/data/rus/"

def read_words(filename):

    if type(filename) == "string":
      with open(filename, 'r') as f:
          return f.read().replace("\n", " <eos> ").replace(".", " .").replace(",", " ,").replace("?", " ?").replace("!", " !").replace(":", " :").split()
    else:
      return filename.replace("\n", " <eos> ").replace(".", " .").replace(",", " ,").replace("?", " ?").replace("!", " !").replace(":", " :").replace(")", " )").replace("(", " (").split()
        
            


def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    #print(count_pairs)
    
    words, _ = list(zip(*count_pairs))
    
    #print(words)
    
    word_to_id = dict(zip(words, range(len(words))))
    
    #print(word_to_id)

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

def load_vk_messages_data(reload = False):

    global allMessages

    vk_data = []

    if reload:

      with open("annLastNN.html", encoding='utf8') as infile:
        parcedMessages = BeautifulSoup(infile, "html.parser")

      allMessages = parcedMessages.body.find_all('div', attrs={'class':'msg_item'})


    allMessagesBatch = allMessages

    for message in allMessagesBatch:

      messageContent = message.find("div", {"class": "msg_body"})
      if messageContent:
        text = messageContent.text
        vk_data.append(text)

    vk_data = " ; ".join(vk_data)

    return vk_data




def load_data(specific_files = False):
    # get the data paths

    if specific_files:

      train_path = specific_files[0]
      valid_path = specific_files[1]
      test_path = specific_files[2]

    else:

      train_path = os.path.join(data_path, "ptb.train.txt")
      valid_path = os.path.join(data_path, "ptb.valid.txt")
      test_path = os.path.join(data_path, "ptb.test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, valid_data, test_data, vocabulary, word_to_id, reversed_dictionary

data_from_vk = load_vk_messages_data(True)

print(len(data_from_vk))

train_data, valid_data, test_data, vocabulary, word_to_id, reversed_dictionary = load_data([data_from_vk[:400000], data_from_vk[400000:500000], data_from_vk[500000:]])

In [ ]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step
        
    def generate(self):
      
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        
        while True:
          
          for i in range(self.batch_size):
              if self.current_idx + self.num_steps >= len(self.data):
                  # reset the index back to the start of the data set
                  self.current_idx = 0
              x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
              temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
              # convert all of temp_y into a one hot representation
              y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
              self.current_idx += self.skip_step
          yield x, y
        
        
num_steps = 10
batch_size = 100
        
train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
        
       


In [ ]:
hidden_size = 500
dropout_rate = 0.7




model = Sequential()

model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))

model.add(CuDNNLSTM(hidden_size, return_sequences=True))
model.add(CuDNNLSTM(hidden_size, return_sequences=True))

model.add(Dropout(dropout_rate))

model.add(Dense(vocabulary))

model.add(Dropout(dropout_rate))

model.add(Dense(vocabulary))

model.add(Activation('softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [ ]:
driveReconnect()

class SaveModelCallback(keras.callbacks.Callback):

    def on_epoch_end(self, batch, logs={}):
        driveReconnect()

saveModelCallback = SaveModelCallback()
checkpointer = ModelCheckpoint(filepath=data_path + '/model-{epoch:02d}.hdf5', verbose=1)
time_print_callback = LambdaCallback(on_epoch_end = lambda _, __: print(datetime.datetime.now()))

In [ ]:
num_epochs = 100

print(len(train_data)//(batch_size*num_steps))

model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[saveModelCallback, time_print_callback, checkpointer])

In [ ]:
driveReconnect()

#name_to_save = "model"
name_to_save = "model_rus_EmLsLsDrDeDrDe"

model_json = model.to_json()

with open(name_to_save + ".json", "w") as json_file:
    json_file.write(model_json)
    
model.save_weights(name_to_save + ".h5")

In [ ]:
driveReconnect()

#name_to_load = "model"
name_to_load = "model_rus_EmLsLsDrDeDrDe"

json_file = open(name_to_load + '.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights(name_to_load + ".h5")

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [ ]:
x_test, _ = next(valid_data_generator.generate())

x_test = x_test[0]
x_test = x_test.reshape((1, num_steps))

print(x_test.shape)

def getAnswer(x_test):

  #print(x_test.shape)

  answer = model.predict_on_batch(x_test)

  def getMax(arr):
    
    maxEl = arr[0]
    maxKey = 0
    
    for key in range(len(arr)):
      
      el = arr[key]
      
      if el > maxEl:
        
        maxEl = el
        maxKey = key
        
    return maxKey

  print(answer.shape)

  answer = answer.reshape(num_steps, 8719)  
  answerMax = []

  for ind, answerForWord in enumerate(answer):

    answerForWord = list(enumerate(answerForWord))
    answerForWord.sort(key = lambda el: el[1], reverse = True)
    answerMax.append(answerForWord[:3])

  x_test = x_test.reshape((num_steps))

  wordsInput = list(map(lambda key : reversed_dictionary[key], x_test))

  answerMaxWords = []

  for ind, answer in enumerate(answerMax):
    answerMaxWords.append([])
    for answerEl in answer:
      answerMaxWords[ind].append(reversed_dictionary[answerEl[0]])

  return wordsInput, answerMaxWords

(wordsInput, wordsAnswer) = getAnswer(x_test)




for ind, el in enumerate(wordsInput):
    print(el.ljust(15, ' ') + " ".join(wordsAnswer[ind]))
